# SQL 


### Описание данных

Нам передали базу данных одного из конкурирующих сервисов. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры на книги. Эти данные помогут **сформулировать ценностное предложение для нового продукта.**




**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # порт подключения
             'db': 'data-analyst-final-project-db'}          # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])


engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

- Исследуйем таблицы (выведите первые строки).

In [3]:
sql = '''select * from books limit 5'''
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
sql = '''select * from authors limit 5'''
select(sql)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
sql = '''select * from publishers limit 5'''
select(sql)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
sql = '''select * from ratings limit 5'''
select(sql)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
sql = '''select * from reviews limit 5'''
select(sql)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

- Посчитаем количество книг, выпущенных после 1 января 2000 года.

In [8]:
sql = '''select count(t.book_id) as books,
                      count(distinct t.book_id) as uniq_books 
         from books t
         where t.publication_date > '2000-01-01' '''
select(sql)

,books,uniq_books
0,819,819


C 1 января 2000 года выпустили **819** книгу.

- Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги.

In [3]:
sql = '''select t.book_id as book,count(distinct t.review_id) as review_count,
                avg(r.rating) as rating_avg
         from reviews t
         inner join ratings r on r.book_id = t.book_id
         where r.book_id = 148
         group by t.book_id
         limit 3
     '''    
select(sql)

,book,review_count,rating_avg
0,148,3,3.428571


In [10]:
pd.io.sql.read_sql('SELECT * FROM reviews where book_id=148;', con=engine)

,review_id,book_id,username,text
0,412,148,znelson,Particular rich run. Lot family positive each.
1,413,148,johnsonamanda,Reach court art probably. Head instead federal...
2,414,148,sabrinabrown,History above dinner strong product it between.


2793 уникальных обзора пользователя и средняя оценка ~3.94.

- Определим издательство, которое издало наибольшее число книг толще 50 страниц (так мы исключите из анализа различные брошюры).

In [11]:
sql = '''select p.publisher as publisher,
                count(t.book_id) as book_cnt,
                count(distinct t.book_id) as uniq_book_id
         from publishers p 
         inner join books t on t.publisher_id = p.publisher_id
         where t.num_pages > 50
         group by p.publisher
         order by uniq_book_id desc
         limit 7
      '''
select(sql)

,publisher,book_cnt,uniq_book_id
0,Penguin Books,42,42
1,Vintage,31,31
2,Grand Central Publishing,25,25
3,Penguin Classics,24,24
4,Ballantine Books,19,19
5,Bantam,19,19
6,Berkley,17,17


Издательство `Penguin Books` издало **42** книги толще 50 страниц, и является лидером среди конкурентов.

- Определим автора с самой высокой средней оценкой книг — учтём только книги с **50 и более** пользовательскими оценками.

In [12]:
sql = ''' select t.author, max(t.rat_avg) as rat_max from
(select s.author as author,avg(r.rating) as rat_avg, count(r.rating_id) as rat_cnt from
(select * from authors a
inner join books b on b.author_id = a.author_id) s
inner join ratings r on r.book_id = s.book_id
group by s.author) t
where t.rat_cnt > 50
group by t.author
order by rat_max desc
limit 5
'''
select(sql)

,author,rat_max
0,J.K. Rowling/Mary GrandPré,4.288462
1,Agatha Christie,4.283019
2,Markus Zusak/Cao Xuân Việt Khương,4.264151
3,J.R.R. Tolkien,4.240964
4,Roald Dahl/Quentin Blake,4.209677


Ровно **50** оценок у **Diana Gabaldon** и самая высокая средняя оценка(4.3) среди книг с **50 и более** пользовательскими оценками.

- Посчитайем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

In [13]:
sql = '''select avg(s.review_count) from (select r.username as username, count(distinct r.review_id) as review_count from reviews r inner join (select t.username as username, count(t.rating_id) as rating_count  from ratings t
group by t.username
having count(t.rating_id) > 50) s on s.username = r.username
group by r.username) s
'''
select(sql)

,avg
0,24.333333


В среднем около **24.333** текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

## Вывод

+ C 1 января 2000 года выпустили **819** книгу;
+ 2793 уникальных обзора пользователя и средняя оценка ~3.94;
+ Издательство `Penguin Books` издало **42** книги толще 50 страниц, и является лидером среди конкурентов;
+ Ровно **50** оценок у **Diana Gabaldon** и самая высокая средняя оценка(4.3) среди книг с **50 и более** пользовательскими оценками;
+ В среднем около **18** текстовых обзоров пользователей, которые поставили более чем по 50 оценок.